In [1]:
#!/usr/bin/env python3

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from secrets import token_bytes
from binascii import hexlify, unhexlify

p = 3548190181
A = 0
B = 0

def invmod(x):
    # secret trick by Fermat
    return pow(x, p-2, p)

def add(P, Q):
    (x1, y1), (x2, y2) = P, Q
    if x1 == x2:
        slope = (3*x1**2 + A) * invmod(2*y1)
    else:
        slope = (y2-y1) * invmod(x2-x1)
    x3 = (slope**2 - x1 - x2) % p
    y3 = (slope*(x1-x3) - y1) % p
    return (x3, y3)

def mul(P, k):
    if k == 1:
        return P
    elif k & 1:
        return add(P, mul(P, k-1))
    else:
        return mul(add(P, P), k//2)

# encrypting stuff

def pkcs7_pad(p, blklen=16):
    n = blklen - (len(p) % blklen)
    return p + bytes([n]*n)

def pkcs7_unpad(p, blklen=16):
    if not len(p) % blklen == 0:
        raise Exception
    last_byte = p[-1]
    pad = bytes([last_byte]*last_byte)
    if last_byte > blklen or last_byte < 1:
        raise Exception
    if not p.endswith(pad):
        raise Exception
    return p[:-last_byte]

def encrypt(p, k):
    iv = token_bytes(16)
    cipher = Cipher(algorithms.AES(k), modes.CBC(iv))
    aes_enc = cipher.encryptor()
    c = aes_enc.update(pkcs7_pad(p)) + aes_enc.finalize()
    return iv + c

def decrypt(c, k):
    iv = c[:16]
    cipher = Cipher(algorithms.AES(k), modes.CBC(iv))
    aes_dec = cipher.decryptor()
    p = aes_dec.update(c[16:]) + aes_dec.finalize()
    return pkcs7_unpad(p)


In [2]:
p = 3548190181

P = (18435, 454929829)
Q = (1254695067, 2137017680)  

In [3]:
print("P")
print("LHS " + str((P[1]**2)%p))
print("RHS " + str((P[0]**3)%p))

P
LHS 2564793410
RHS 2564793410


In [4]:
print("Q")
print("LHS " + str((Q[1]**2)%p))
print("RHS " + str((Q[0]**3)%p))

Q
LHS 946044702
RHS 946044702


In [5]:
# Q == d*P
# d = 1
d = 42069
while True:
    if Q == mul(P, d):
        print("d: " + str(d))
        break
    d += 1

d: 42069


In [6]:
# key = SHA256(str(d))
import hashlib
# key = b'3f64a7f4dd53bb20cfbc148d34d8bf866c5f8728da9dce2066be0e3e408c612e'
key = hashlib.sha256(str(d).encode('utf-8')).digest()

In [7]:
from base64 import b64encode
with open("./flag.pdf.enc", "rb") as file:
    s = file.read()
    decrypted_string = decrypt(s, key)
    decfile = open(f'./flag.pdf', 'wb')
    decfile.write(decrypted_string)
    decfile.close()